# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Oct1722")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL4-FI10-#1_CRR_DiadFit,103.405375,0.011434,0.008295,1284.919544,0.007028,584.258712,1284.919544,1085.072537,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL4-FI10-#10_CRR_DiadFit,103.411159,0.032363,0.024806,1284.907780,0.022951,146.806521,1284.907780,270.460933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL4-FI10-#11,103.390954,0.011602,0.008604,1284.960255,0.007634,513.356229,1284.960255,888.803534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LL4-FI10-#12_CRR_DiadFit,103.422823,0.016333,0.013657,1284.886281,0.013320,475.535121,1284.886281,873.118382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,LL4-FI10-#13_CRR_DiadFit,103.410958,0.008635,0.008635,1284.888440,0.008635,374.531553,1284.888440,680.925579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL4-FI10-#1.txt',
 'LL4-FI10-#10.txt',
 'LL4-FI10-#11.txt',
 'LL4-FI10-#12.txt',
 'LL4-FI10-#13.txt',
 'LL4-FI10-#2.txt',
 'LL4-FI10-#3.txt',
 'LL4-FI10-#4.txt',
 'LL4-FI10-#5.txt',
 'LL4-FI10-#6.txt',
 'LL4-FI10-#7.txt',
 'LL4-FI10-#8.txt',
 'LL4-FI10-#9.txt',
 'LL4-FI12-#1.txt',
 'LL4-FI12-#2.txt',
 'LL4-FI12dp.txt',
 'LL4-FI5-rep1.txt',
 'LL4-FI6.txt',
 'LL4-FI7-#1.txt',
 'LL4-FI7-#10-rep1.txt',
 'LL4-FI7-#10.txt',
 'LL4-FI7-#11.txt',
 'LL4-FI7-#12.txt',
 'LL4-FI7-#12cosmicray.txt',
 'LL4-FI7-#13.txt',
 'LL4-FI7-#2.txt',
 'LL4-FI7-#3-cosmicrays.txt',
 'LL4-FI7-#3.txt',
 'LL4-FI7-#4.txt',
 'LL4-FI7-#5.txt',
 'LL4-FI7-#6.txt',
 'LL4-FI7-#7-rep1.txt',
 'LL4-FI7-#7.txt',
 'LL4-FI7-#8.txt',
 'LL4-FI7-#9.txt',
 'LL4-FI8-#1.txt',
 'LL4-FI8-#1cosmicratondiad.txt',
 'LL4-FI8-stitch.txt',
 'LL4-FI8-stitchHQ.txt',
 'LL4-FI9-#1.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 40/40 [00:07<00:00,  5.62it/s]

Done


'LL4-FI5-rep1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'LL4-FI5-rep1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['LL4-FI10-#1', 'LL4-FI10-#10', 'LL4-FI10-#11', 'LL4-FI10-#12',
       'LL4-FI10-#13', 'LL4-FI10-#2', 'LL4-FI10-#3', 'LL4-FI10-#4',
       'LL4-FI10-#5', 'LL4-FI10-#6', 'LL4-FI10-#7', 'LL4-FI10-#8',
       'LL4-FI10-#9', 'LL4-FI12-#1', 'LL4-FI12-#2', 'LL4-FI12dp',
       'LL4-FI6', 'LL4-FI7-#1', 'LL4-FI7-#10-rep1', 'LL4-FI7-#11',
       'LL4-FI7-#12', 'LL4-FI7-#12cosmicray', 'LL4-FI7-#13', 'LL4-FI7-#2',
       'LL4-FI7-#3-cosmicrays', 'LL4-FI7-#3', 'LL4-FI7-#4', 'LL4-FI7-#5',
       'LL4-FI7-#6', 'LL4-FI7-#7-rep1', 'LL4-FI7-#7', 'LL4-FI7-#8',
       'LL4-FI7-#9', 'LL4-FI9-#1', 'LL4-FI5-rep1', 'LL4-FI7-#10',
       'LL4-FI8-#1', 'LL4-FI8-#1cosmicratondiad'], dtype='<U25')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


2


,time,preferred_values,lower_values,upper_values
0,53292.0,0.997729,0.997725,0.997734
1,57603.0,0.997707,0.997702,0.997712
2,58078.0,0.997705,0.997700,0.997710
3,58477.0,0.997704,0.997699,0.997709
4,59053.0,0.997702,0.997697,0.997707


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name_x,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name_y
0,LL4-FI10-#1.txt,1150.268124,375.595524,229.309453,Spline,NaN,NaN,NaN,NaN
1,LL4-FI10-#10.txt,1150.179632,66.344330,48.508627,Spline,NaN,NaN,NaN,NaN
2,LL4-FI10-#11.txt,1150.268124,227.124122,148.189523,Spline,NaN,NaN,NaN,NaN
3,LL4-FI10-#12.txt,1150.297622,230.768777,154.467402,Spline,NaN,NaN,NaN,NaN
4,LL4-FI10-#13.txt,1150.268124,194.682113,125.719985,Spline,NaN,NaN,NaN,NaN


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [15]:
file_sec_phase

array(['LL4-FI10-#1', 'LL4-FI10-#10', 'LL4-FI10-#11', 'LL4-FI10-#12',
       'LL4-FI10-#13', 'LL4-FI10-#2', 'LL4-FI10-#3', 'LL4-FI10-#4',
       'LL4-FI10-#5', 'LL4-FI10-#6', 'LL4-FI10-#7', 'LL4-FI10-#8',
       'LL4-FI10-#9', 'LL4-FI12-#1', 'LL4-FI12dp', 'LL4-FI5-rep1',
       'LL4-FI6', 'LL4-FI7-#10-rep1', 'LL4-FI7-#10', 'LL4-FI7-#11',
       'LL4-FI7-#12', 'LL4-FI7-#12cosmicray', 'LL4-FI7-#2',
       'LL4-FI7-#3-cosmicrays', 'LL4-FI7-#3', 'LL4-FI7-#4', 'LL4-FI7-#5',
       'LL4-FI7-#6', 'LL4-FI7-#8', 'LL4-FI7-#9', 'LL4-FI8-#1',
       'LL4-FI8-#1cosmicratondiad', 'LL4-FI9-#1'], dtype=object)

In [16]:
df_combo['name_for_matching']

0                   LL4-FI10-#1
1                  LL4-FI10-#10
2                  LL4-FI10-#11
3                  LL4-FI10-#12
4                  LL4-FI10-#13
5                   LL4-FI10-#2
6                   LL4-FI10-#3
7                   LL4-FI10-#4
8                   LL4-FI10-#5
9                   LL4-FI10-#6
10                  LL4-FI10-#7
11                  LL4-FI10-#8
12                  LL4-FI10-#9
13                  LL4-FI12-#1
14                  LL4-FI12-#2
15                   LL4-FI12dp
16                      LL4-FI6
17                   LL4-FI7-#1
18             LL4-FI7-#10-rep1
19                  LL4-FI7-#11
20                  LL4-FI7-#12
21         LL4-FI7-#12cosmicray
22                  LL4-FI7-#13
23                   LL4-FI7-#2
24        LL4-FI7-#3-cosmicrays
25                   LL4-FI7-#3
26                   LL4-FI7-#4
27                   LL4-FI7-#5
28                   LL4-FI7-#6
29              LL4-FI7-#7-rep1
30                   LL4-FI7-#7
31      

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name_x,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name_y,Carb_Diad_Ratio,SO2_Diad_Ratio
0,0,103.170590,103.178881,103.162299,0.008291,LL4-FI10-#1_CRR_DiadFit,103.405375,0.011434,0.008295,1284.919544,...,1150.268124,375.595524,229.309453,Spline,NaN,NaN,NaN,NaN,NaN,0.136573
1,0,103.174010,103.198764,103.149255,0.024755,LL4-FI10-#10_CRR_DiadFit,103.411159,0.032363,0.024806,1284.907780,...,1150.179632,66.344330,48.508627,Spline,NaN,NaN,NaN,NaN,NaN,0.100906
2,0,103.153681,103.162280,103.145082,0.008599,LL4-FI10-#11,103.390954,0.011602,0.008604,1284.960255,...,1150.268124,227.124122,148.189523,Spline,NaN,NaN,NaN,NaN,NaN,0.101527
3,0,103.185352,103.198987,103.171717,0.013635,LL4-FI10-#12_CRR_DiadFit,103.422823,0.016333,0.013657,1284.886281,...,1150.297622,230.768777,154.467402,Spline,NaN,NaN,NaN,NaN,NaN,0.106145
4,0,103.173363,103.181994,103.164733,0.008630,LL4-FI10-#13_CRR_DiadFit,103.410958,0.008635,0.008635,1284.888440,...,1150.268124,194.682113,125.719985,Spline,NaN,NaN,NaN,NaN,NaN,0.116977


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [20]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()